# Genetic Algorithms example: Symbolic Regression


In [ ]:
!pip install gplearn
!pip install pydotplus

In [ ]:
from gplearn.genetic import SymbolicRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.neural_network import MLPRegressor
from sklearn.utils.random import check_random_state
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import numpy as np
from IPython.display import Image
import pydotplus

First, we'll define the function we want to fit:

In [ ]:
x0 = np.arange(-1, 1, 0.1)
x1 = np.arange(-1, 1, 0.1)
x0, x1 = np.meshgrid(x0, x1)

def y(x0, x1):
    return (x0 ** 2) + np.sin(x1) -1

y_truth = y(x0, x1)

What does the function look like?

In [ ]:
ax = plt.figure().gca(projection='3d')
ax.set_xlim(-1, 1)
ax.set_ylim(-1,1)
surf = ax.plot_surface(x0, x1, y_truth, rstride=2, cstride=2,
                       color='green', alpha=0.5)
plt.show()

For training, we'll generate some more data to train on

In [ ]:
train_size = 100
test_size = 50

X_train = np.random.rand(train_size, 2) * 2 - 1
y_train = y(X_train[:, 0], X_train[:, 1])

X_test = np.random.rand(test_size, 2) * 2 - 1
y_test = y(X_test[:, 0], X_test[:, 1])

X_train.shape

Now we can go on to define the functions allowed for use by the regressor and define it!

In [ ]:
function_set = ['add', 'sub', 'mul', 'div',
                'sqrt', 'log', 'abs', 'neg', 'inv',
                'max', 'min', 'sin', 'cos', 'log']

genetic_regressor = SymbolicRegressor(population_size=5000,
                                generations=5, 
                              stopping_criteria=0.01,
                                p_crossover=0.7, p_subtree_mutation=0.1,
                                p_hoist_mutation=0.05, p_point_mutation=0.1,
                                max_samples=0.9, verbose=1,
                                function_set=function_set,
                                parsimony_coefficient=0.01)

Let's train:

In [ ]:
genetic_regressor.fit(X_train, y_train)

GPLearn allows us to visualize the regressor:

(You must install graphviz to see it)
```
sudo apt-get install graphviz
```

In [ ]:
graph = pydotplus.graphviz.graph_from_dot_data(genetic_regressor._program.export_graphviz())
Image(graph.create_png())

Now that we have the regressor, we can also see how it came to be.

In [ ]:
print(genetic_regressor._program.parents)

In [ ]:
parent_id = genetic_regressor._program.parents['parent_idx']
graph = genetic_regressor._programs[-2][parent_id].export_graphviz()
graph = pydotplus.graphviz.graph_from_dot_data(graph)
Image(graph.create_png())

Before we go on to measure the accuracy of our regressor, we'll train a couple more with different methods in order to compare their performances:

A decision tree:

In [ ]:
tree_regressor = DecisionTreeRegressor()
tree_regressor.fit(X_train, y_train)

And a neural network:

In [ ]:
neural_regressor = MLPRegressor(verbose=True,
                                hidden_layer_sizes=(10, 10),
                               learning_rate_init=0.01,
                               max_iter=100,
                               tol=-100)
neural_regressor.fit(X_train, y_train)

All three regressors will try and evaluate the function over our space:

In [ ]:
y_genetic = genetic_regressor.predict(np.c_[x0.ravel(), x1.ravel()]).reshape(x0.shape)
score_genetic = genetic_regressor.score(X_test, y_test)

y_tree = tree_regressor.predict(np.c_[x0.ravel(), x1.ravel()]).reshape(x0.shape)
score_tree = tree_regressor.score(X_test, y_test)

y_neural = neural_regressor.predict(np.c_[x0.ravel(), x1.ravel()]).reshape(x0.shape)
score_neural = neural_regressor.score(X_test, y_test)

Finally, let's compare what they've drawn!

In [ ]:
fig = plt.figure(figsize=(12, 10))

for i, (y_pred, score, title) in enumerate([(y_truth, None, "Ground Truth"),
                                       (y_genetic, score_genetic, "GeneticRegressor"),
                                       (y_tree, score_tree, "DecisionTreeRegressor"),
                                       (y_neural, score_neural, "NeuralRegressor"),

                                      ]):

    ax = fig.add_subplot(2, 2, i+1, projection='3d')
    ax.set_xlim(-1, 1)
    ax.set_ylim(-1, 1)
    surf = ax.plot_surface(x0, x1, y_pred, rstride=1, cstride=1, color='green', alpha=0.5)
    points = ax.scatter(X_train[:, 0], X_train[:, 1], y_train)
    if score is not None:
        score = ax.text(-.7, 1, .2, "$R^2 =\/ %.6f$" % score, 'x', fontsize=14)
    plt.title(title)
plt.show()